In [41]:
from sklearn.cluster import KMeans
# common_text에는 파싱된 워드 리스트들이 들어가 있음.
from gensim.test.utils import common_texts
# Doc2Vec이 우리가 텍스트를 사용해서 학습되는 모델(뉴럴넷)이고 
# TaggedDocument가 넘겨주는 텍스트들. 
# 여기서, corpus와 ID들을 함께 넘겨줘야 하는데, 여기서 ID는 tag와 같은 말임
# Q: 여기서, 여러 tag를 함께 넘겨준 다음, 적합한 태그를 찾아주는 방식으로도 처리할 수 있는지 파악하는 것이 필요함. 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# import하기
import json
  
# Opening JSON file
f = open('key_as_space.json')  
awesome_file = json.load(f)

# TaggedDocument는 해당 corpus의 id를 함께 넘겨주는 것을 말합니다. 
# 아래 코드에서 doc에는 단어의 묶음이, tags에는 해당 문서를 표현하는 태그가 들어가게 됨. 
# 흠, 그렇다면, 이 태그에 고유 id가 아닌 다른 것을 넣어주면 현재 문서에 맞는 tag를 찾아주기도 하나? 
# 이렇게 할 경우, words에는 문서를, tags에는 키워드를 넣고, words에서 tags를 예측하는 짓을 할 수도 있지 않을까? 

common_texts_and_tags = [
    (value_array, key) for key, value_array in awesome_file.items()
]



# print("##"*20)
# print("tags and its texts")
# print("##"*20)
# for text, tag in common_texts_and_tags:
#     print(f"tag: {tag}, text: {text}")
#     print("\n")

# text: 단어별로 분할되어 있는 리스트 
# tags: 해당 문서를 의미하는 태그. 여기서, tags를 unique id로 넣기는 했는데, 그렇지 않게 넣어서 학습시키는 것도 가능할 것으로 보임. 
TRAIN_documents = [TaggedDocument(words=text, tags=[tag]) for text, tag in common_texts_and_tags]
# training. 
# 여러 Parameter들을 사용하여 튜닝할 수 있음. 
model = Doc2Vec(TRAIN_documents, vector_size=100, window=100, epochs=1000, min_count=0, workers=4)
model.build_vocab(TRAIN_documents)


In [43]:
print("##"*20)
print("Accessing Document vector")
print("##"*20)
# # document의 vector를 구하는 방법은, 기 학습된 docvec을 가져오거나, 해당 단어를 넣고 다시 예측하는 것. 
# # 단, tag로 접근이 가능할 뿐, 학습한 doc를 그대로 가져오는 것은 어려운 것으로 보임. 
# # 또한, 여기서, 학습된 vector와 inferring한 vector가 미묘하게 다른데, 이것은 현재 train set이 적어서 그런 것으로 보임. 
for text, tags in common_texts_and_tags:
    trained_doc_vec = model.docvecs[tags]
    inferred_doc_vec = model.infer_vector(text)
    print(f"tags: {tags}") #, text: {text}")
    print(f"trained_doc_vec: {trained_doc_vec}")
    print(f"inferred_doc_vec: {inferred_doc_vec}")
    print("--"*20)
    
print("##"*20)
print("predicting Document vector")
print("##"*20)

########################################
Accessing Document vector
########################################


/var/folders/ll/f1h284rn02128q30qhbmyqn80000gn/T/ipykernel_5065/3266457395.py:8: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags]


tags: user interface components
trained_doc_vec: [-24.753946    1.1205219 -81.00152    12.573917    1.0861676]
inferred_doc_vec: [ 0.0146185  -0.02399996  0.01639508 -0.09207737  0.0692867 ]
----------------------------------------
tags: security
trained_doc_vec: [ 17.426928  -53.029297  -40.39464   -47.49701    -6.2233505]
inferred_doc_vec: [-0.07470004  0.05900669  0.04503072  0.03770196 -0.03894429]
----------------------------------------
tags: machine learning
trained_doc_vec: [ -2.795391   58.98972   -37.28411   -42.114197   -6.0221143]
inferred_doc_vec: [ 0.05930215 -0.05134643  0.01757727 -0.01563345 -0.05074674]
----------------------------------------
tags: networking
trained_doc_vec: [-15.284109  28.927063 -10.542066 -68.70677  -24.78545 ]
inferred_doc_vec: [-0.03497486  0.06337665  0.01504725 -0.02833132 -0.06709182]
----------------------------------------
tags: user interface
trained_doc_vec: [-26.204777    4.2509885 -55.7311    -31.879868   36.923927 ]
inferred_doc_vec: 

In [48]:
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import word_tokenize
# python3 -c "import nltk; nltk.download('all')"

#to find the vector of a document which is not in training data
test_data = word_tokenize("This is a small arcade game MVP where the objective is to survive for as long as possible dodging space debris".lower())
print(model.dv.most_similar(positive=[model.infer_vector(test_data)],topn=5))

[('software development', 0.6700701713562012), ('graphics', 0.588083803653717), ('software performance', 0.5489650964736938), ('community', 0.5440669655799866), ('web browsers', 0.5039184093475342)]


In [49]:
test_data = word_tokenize("This project serves as the backend of a mobile application Forword. The backend is developed in the form of a RESTAPI.")
print(model.dv.most_similar(positive=[model.infer_vector(test_data)],topn=5))

[('cloud computing', 0.42688634991645813), ('operating systems', 0.3926736116409302), ('advertising', 0.33251315355300903), ('media', 0.30980008840560913), ('collaboration', 0.23913080990314484)]


In [ ]:

# document clustering 

print("##"*30)
print("K-Means Clustering")
print("##"*30)
Clustering_Method = KMeans(n_clusters=2, random_state=0)
X = model.docvecs.vectors_docs # document vector 전체를 가져옴. 
Clustering_Method.fit(X)# fitting 
# 결과를 보면 알겠지만, 생각보다 클러스터링이 잘 되지 않음. 
# 일단은 이것 또한 트레이닝 셋이 적어서 그런 것으로 보임. 
cluster_dict = {i:[] for i in range(0, 2)}
for text_tags, label in zip(common_texts_and_tags, Clustering_Method.labels_):
    text, tags = text_tags
    cluster_dict[label].append(text)
for label, lst in cluster_dict.items():
    print(f"Cluster {label}")
    for x in lst:
        print(x)
    print("--"*30)
print("##"*20)